In [ ]:
!pip install ultralytics
!pip install easyocr
!pip install opencv-python

import os
import cv2
import math
import easyocr
from ultralytics import YOLO
from collections import Counter
import matplotlib.pyplot as plt

!pip install -q streamlit opencv-python-headless easyocr matplotlib ultralytics
!npm install -g localtunnel



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Detection Model

In [11]:
import os
import cv2
import math
import easyocr
from ultralytics import YOLO
from collections import Counter
import matplotlib.pyplot as plt

# --- Configuration ---
MODEL_PATHS = ['/content/drive/MyDrive/finalbest.pt']
SECONDARY_MODEL_PATH = '/content/drive/MyDrive/tele_sw_best.pt'
EARTH_MODEL_PATH = '/content/drive/MyDrive/best_earth.pt'

REQUIRED_CLASSES = [
    'battery', 'bluethooth', 'clock', 'diode', 'dot', 'dwarrow', 'hazzard', 'magnet', 'redl',
    'ruppee', 'sdarrow', 'sym_T', 'tele', 'tempearature', 'uparrow',
    'bill', 'byp', 'cum', 'date', 'digit', 'kwarhz', 'last', 'md', 'mdm',
    'net', 'nmnd', 'pf', 'pp', 'rev', 'smdigit', 'switch', 'time', 'top', 'wifi', 'earth'
]
TEXT_CHECK_CLASSES = {
    'bill', 'byp', 'cum', 'date', 'last', 'md', 'mdm', 'net', 'nmnd',
    'pf', 'pp', 'rev', 'earth', 'time', 'top'
}
THIRD_STAGE_CLASSES = {'smdigit', 'tele', 'switch'}

# --- Initialize models ---
reader = easyocr.Reader(['en'], gpu=True)
models = [YOLO(path) for path in MODEL_PATHS]
secondary_model = YOLO(SECONDARY_MODEL_PATH)
earth_model = YOLO(EARTH_MODEL_PATH)

def read_and_preprocess_image(image_path):
    return cv2.imread(image_path)

def save_crop(image, box, output_folder, class_name, index):
    x1, y1, x2, y2 = box
    if class_name == 'kwarhz':
        crop = image[y1-6:y2+10, x1-2:x2+2]
    else:
        crop = image[y1:y2, x1:x2]
    filename = os.path.join(output_folder, f"{class_name}_{index}.png")
    cv2.imwrite(filename, crop)
    return crop

def run_detection(image):
    detections, detected_classes = [], []
    for model in models:
        results = model(image)
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0].item())
                class_name = model.names[cls_id]
                conf = box.conf[0].item()
                xyxy = list(map(int, box.xyxy[0].tolist()))
                detections.append({
                    'class': class_name,
                    'box': xyxy,
                    'conf': conf
                })
                detected_classes.append(class_name)
    return detections, detected_classes

def custom_crop_for_ocr(image, box, cls, img_height, img_width):
    x1, y1, x2, y2 = box
    h, w = y2 - y1, x2 - x1

    if cls in {'mdm', 'top'}:
        y2 +=16
        x1-=5
    elif cls == 'earth':
        y2 = y1 + int(h * 0.65)

        x2 += 3
    elif cls == 'pp':
        x2 = min(x2 + 2, img_width)
    elif cls == 'rev':
        y2 = y1 + int(h * 0.65)
        x2 +=2

    return image[y1:y2, x1:x2]

def get_ocr_text(image_crop):
    gray = cv2.cvtColor(image_crop, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    return ''.join([res[1] for res in result]).replace("'", "").replace(" ", "").lower()

def run_secondary_and_count(crop):
    results = secondary_model(crop)
    counts = Counter()
    for r in results:
        for box in r.boxes:
            class_idx = int(box.cls[0].item())
            name = secondary_model.names[class_idx]
            counts[name] += 1
    return counts

def run_earth_model(crop):
    results = earth_model(crop)
    counts = Counter()
    for r in results:
        for box in r.boxes:
            class_idx = int(box.cls[0].item())
            name = earth_model.names[class_idx]
            counts[name] += 1
    return counts

# ---- Main Function: run_defect_detection ----
def run_defect_detection(image_path, output_folder='crops'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image = read_and_preprocess_image(image_path)
    height, width = image.shape[:2]

    detections, detected_classes = run_detection(image)

    for idx, det in enumerate(detections):
        det['crop'] = save_crop(image, det['box'], output_folder, det['class'], idx)

    # --- Stage 1 ---
    counts = Counter(detected_classes)
    stage1_errors = []
    for cls in REQUIRED_CLASSES:
        count = counts.get(cls, 0)
        if cls == 'redl' and count != 2:
            stage1_errors.append(f"Class '{cls}' has {count} detections (expected exactly 2)")
        elif cls == 'digit' and count < 8:
            stage1_errors.append(f"Class '{cls}' has {count} detections (expected at least 8)")
        elif cls == 'dot' and count < 9:
            stage1_errors.append(f"Class '{cls}' has {count} detections (expected at least 9)")
        elif cls not in ['digit', 'dot', 'redl'] and count < 1:
            stage1_errors.append(f"Class '{cls}' is missing")

    # --- Stage 2 ---
    stage2_errors = []
    ocr_by_class = {cls: [] for cls in TEXT_CHECK_CLASSES}
    for det in detections:
        cls = det['class']
        if cls in TEXT_CHECK_CLASSES:
            crop = custom_crop_for_ocr(image, det['box'], cls, height, width)
            text = get_ocr_text(crop)
            ocr_by_class[cls].append(text)


    for cls, texts in ocr_by_class.items():
      if not any(
          cls in t or
          (cls == 'top' and t in {'top', 't0p'}) or
          (cls == 'earth' and t in {'earth', 'eacth'})
          for t in texts
      ):
        stage2_errors.append(f"OCR mismatch for class '{cls}': none matched, found {texts}")

    # --- Stage 3 ---
    stage3_errors = []
    for name, checks in {
        'smdigit': lambda c: c.get('smseg', 0) >= 14 and c.get('smdot', 0) >= 1,
        'tele': lambda c: c.get('tower', 0) >= 4,
        'switch': lambda c: all(c.get(k, 0) >= 1 for k in ['swl', 'swr', 'swb']),
    }.items():
        if any(d['class'] == name for d in detections):
            if not any(checks(run_secondary_and_count(d['crop'])) for d in detections if d['class'] == name):
                stage3_errors.append(f"'{name}' failed: Missing Subclass")

    # --- Stage 4 ---
    stage4_errors = []
    for idx, det in enumerate(d for d in detections if d['class'] == 'digit'):
        gray = cv2.cvtColor(det['crop'], cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        seg_coords = {
            'A': (h//3 - 3, w - 7), 'B': (h//3*2 + 3, w - 7), 'C': (h - 4, w // 2),
            'D': (h//3*2 + 3, 7), 'E': (h//3 - 3, 7), 'F': (4, w // 2), 'G': (h // 2, w // 2)
        }
        failed = [s for s, (y, x) in seg_coords.items() if gray[y, x] > 120]
        if failed:
            stage4_errors.append(f"Digit #{idx+1} segment check failed: segments {failed} have pixel > 120")

    # --- Stage 5 ---
    stage5_errors = []
    for det in detections:
        if det['class'] == 'kwarhz':
            crop = det['crop']
            h, w = crop.shape[:2]
            h = h//2
            w = w//2
            kw = crop[:, :math.ceil(w - w*0.09)]
            a = crop[:, math.ceil(w - w*0.255):]
            combined_text = (get_ocr_text(kw) + get_ocr_text(a)).lower()
            '''
            plt.subplot(1, 2, 1)
            plt.imshow(kw, cmap='gray')
            plt.axis('on')
            plt.title("KW Part")

            plt.subplot(1, 2, 2)
            plt.imshow(a, cmap='gray')
            plt.axis('on')
            plt.title("Arhz Part")
            '''
            if 'kwarhz' not in combined_text:
                stage5_errors.append(f"OCR mismatch for 'kwarhz': got '{combined_text}'")

    # --- Stage 6 ---
    stage6_errors = []
    earth_detections = [d for d in detections if d['class'] == 'earth']
    if earth_detections:
        if not any(run_earth_model(d['crop']).get('underline', 0) >= 3 for d in earth_detections):
            stage6_errors.append("'earth' defective: none of the earth crops had at least 3 underline detections")

    return {
        'stage1_errors': stage1_errors,
        'stage2_errors': stage2_errors,
        'stage3_errors': stage3_errors,
        'stage4_errors': stage4_errors,
        'stage5_errors': stage5_errors,
        'stage6_errors': stage6_errors
    }


if __name__ == "__main__":
    result = run_defect_detection("/content/test12.PNG")

    print("\n--- Stage 1: Missing or Count Errors ---")
    for err in result['stage1_errors']:
        print(err)

    print("\n--- Stage 2: OCR Mismatch Errors ---")
    for err in result['stage2_errors']:
        print( err)

    print("\n--- Stage 3: Subclass Errors ---")
    for err in result['stage3_errors']:
        print(err)

    print("\n--- Stage 4: 7-Segment digit Intensity Errors ---")
    for err in result['stage4_errors']:
        print(err)

    print("\n--- Stage 5: KWARHZ OCR Errors ---")
    for err in result['stage5_errors']:
        print(err)

    print("\n--- Stage 6: Earth Underline Detection Errors ---")
    for err in result['stage6_errors']:
        print(err)



0: 640x864 1 battery, 1 bill, 1 bluethooth, 1 byp, 1 clock, 1 cum, 1 date, 7 digits, 1 diode, 9 dots, 1 dwarrow, 1 earth, 1 hazzard, 1 kwarhz, 1 magnet, 1 md, 1 mdm, 1 net, 1 nmnd, 1 pf, 1 pp, 2 redls, 1 rev, 1 ruppee, 1 sdarrow, 4 smdigits, 1 switch, 1 sym_T, 1 tele, 1 tempearature, 1 time, 1 top, 1 uparrow, 1 wifi, 49.3ms
Speed: 5.3ms preprocess, 49.3ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 864)

0: 800x864 14 smsegs, 70.2ms
Speed: 5.1ms preprocess, 70.2ms inference, 1.9ms postprocess per image at shape (1, 3, 800, 864)

0: 864x736 10 smsegs, 68.6ms
Speed: 4.6ms preprocess, 68.6ms inference, 1.8ms postprocess per image at shape (1, 3, 864, 736)

0: 864x544 5 smsegs, 69.6ms
Speed: 3.4ms preprocess, 69.6ms inference, 2.0ms postprocess per image at shape (1, 3, 864, 544)

0: 864x448 5 smsegs, 70.3ms
Speed: 2.9ms preprocess, 70.3ms inference, 2.0ms postprocess per image at shape (1, 3, 864, 448)

0: 864x864 4 towers, 38.1ms
Speed: 5.4ms preprocess, 38.1ms inference

In [ ]:
import shutil
import os

folder_path = '/content/crops'
#folder_path = '/content/img'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f" Deleted folder: {folder_path}")
else:
    print(f" Folder does not exist: {folder_path}")

In [ ]:
import os
import cv2
import easyocr
from ultralytics import YOLO
from collections import Counter
import matplotlib.pyplot as plt

# --- Configuration ---
model_paths = ['/content/drive/MyDrive/finalbest.pt']
secondary_model_path = '/content/drive/MyDrive/tele_sw_best.pt'

required_classes = [
    'battery', 'bluethooth', 'clock', 'diode', 'dot', 'dwarrow', 'hazzard', 'magnet', 'redl',
    'ruppee', 'sdarrow', 'sym_T', 'tele', 'tempearature', 'uparrow',
    'bill', 'byp', 'cum', 'date', 'digit', 'kwarhz', 'last', 'md', 'mdm',
    'net', 'nmnd', 'pf', 'pp', 'rev', 'smdigit', 'switch', 'time', 'top', 'wifi', 'earth'
]

text_check_classes = {
    'bill', 'byp', 'cum', 'date', 'last', 'md', 'mdm', 'net', 'nmnd',
    'pf', 'pp', 'rev', 'earth', 'time', 'top'
}

third_stage_classes = {'smdigit', 'tele', 'switch'}

reader = easyocr.Reader(['en'], gpu=True)

# --- Load YOLO Models ---
models = [YOLO(path) for path in model_paths]
secondary_model = YOLO(secondary_model_path)

def run_defect_detection(image_path, output_folder='crops'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image = cv2.imread(image_path)
    height, width = image.shape[:2]
    detections = []
    detected_classes = []

    for model in models:
        results = model(image)
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0].item())

                class_name = model.names[cls_id]
                #print(class_name)
                conf = box.conf[0].item()
                xyxy = list(map(int, box.xyxy[0].tolist()))
                x1, y1, x2, y2 = xyxy
                if class_name == 'kwarhz':

                  crop = image[y1-6:y2+10, x1-2:x2+2]
                  '''
                  plt.subplot(1, 2, 1)
                  plt.imshow(crop, cmap='gray')
                  plt.axis('on')
                  plt.title("KW Part")
                  '''
                else:
                  crop = image[y1:y2, x1:x2]
                crop_filename = os.path.join(output_folder, f"{class_name}_{len(detections)}.png")
                cv2.imwrite(crop_filename, crop)
                detections.append({'class': class_name, 'box': [x1, y1, x2, y2], 'conf': conf,'crop':crop})
                detected_classes.append(class_name)

    # --- Stage 1: Count validation ---
    counts = dict(Counter(detected_classes))
    class_counts = {cls: counts.get(cls, 0) for cls in required_classes}
    stage1_errors = []

    for cls in required_classes:
        count = class_counts[cls]
        if cls == 'redl':
            if count != 2:
                stage1_errors.append(f"Class '{cls}' has {count} detections (expected exactly 2)")
        elif cls == 'digit':
            if count < 8:
                stage1_errors.append(f"Class '{cls}' has {count} detections (expected at least 8)")
        elif cls == 'dot':
            if count < 9:
                stage1_errors.append(f"Class '{cls}' has {count} detections (expected at least 9)")
        else:
            if count < 1:
                stage1_errors.append(f"Class '{cls}' is missing")

    # --- Stage 2: OCR validation with class-specific cropping and matching ---
    stage2_errors = []
    img_height, img_width = image.shape[:2]
    class_ocr_texts = {cls: [] for cls in text_check_classes}

    for det in detections:
        cls = det['class']
        if cls in text_check_classes:
            x1, y1, x2, y2 = det['box']
            h = y2 - y1
            w = x2 - x1

            if cls in {'mdm', 'top'}:
                y2+=16
                x1-=5
                cropped = image[y1:y2, x1:x2]

            elif cls == 'earth':
                y2_cropped = y1 + int(h * 0.65)
                x2+=3
                cropped = image[y1:y2, x1:x2]

            elif cls == 'pp':
                x2_padded = min(x2 + 2, img_width)
                cropped = image[y1:y2, x1:x2_padded]
            elif cls == 'rev':
                y2_cropped = y1 + int(h * 0.5)
                x2+=2
                cropped = image[y1:y2_cropped, x1:x2]
            else:
                cropped = image[y1:y2, x1:x2]




            gray_crop = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
            ocr_result = reader.readtext(gray_crop)
            text_cleaned = ''.join([res[1] for res in ocr_result]).lower()
            text_nospaces = ''.join(text_cleaned.replace("'", "").split())
            class_ocr_texts[cls].append(text_nospaces)

    for cls, texts in class_ocr_texts.items():
        match_found = False
        for text in texts:

            if cls == 'top':
                if text in {'top', 't0p'} or cls in text :
                    match_found = True
                    break
            elif cls in text:
                match_found = True
                break
        if not match_found:
            stage2_errors.append(f"OCR mismatch for class '{cls}': none matched, found {texts}")

    # --- Stage 3: Run third model on cropped smdigit, tele, switch ---
    stage3_errors = []
    third_stage_crops = [d for d in detections if d['class'] in third_stage_classes]


    stage4_errors = []
    digit_detections = [d for d in detections if d['class'] == 'digit']

    for idx, det in enumerate(digit_detections):
        x1, y1, x2, y2 = det['box']
        crop = image[y1:y2, x1:x2]
        gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape[:2]

        segment_coords = {
            'A': (h//3 - 3, w - 7),
            'B': (h//3*2 + 3, w - 7),
            'C': (h - 4, w // 2),
            'D': (h//3*2 + 3, 7),
            'E': (h//3 - 3, 7),
            'F': (4, w // 2),
            'G': (h // 2, w // 2),
        }

        failed_segments = []
        for seg, (yy, xx) in segment_coords.items():
            if 0 <= yy < h and 0 <= xx < w:
                if gray[yy, xx] > 120:
                    failed_segments.append(seg)

        if failed_segments:
            stage4_errors.append(
                f"Digit #{idx+1} segment check failed: segments {failed_segments} have pixel > 100"
            )

    # --- Stage 3: Secondary Validation for smdigit, tele, switch ---
    stage3_errors = []

    def run_secondary_and_count(crop):
        results = secondary_model(crop)
        counts = Counter()
        for r in results:
            for box in r.boxes:
                class_idx = int(box.cls[0].item())
                name = secondary_model.names[class_idx]

                counts[name] += 1
        return counts

    # --- smdigit ---
    smdigit_success = False
    for det in [d for d in detections if d['class'] == 'smdigit']:
        x1, y1, x2, y2 = det['box']
        crop = image[y1:y2+2, x1:x2+2]
        counts = run_secondary_and_count(crop)

        if counts.get('smseg', 0) >= 14 and counts.get('smdot', 0) >= 1:
            smdigit_success = True
            break

    if any(d['class'] == 'smdigit' for d in detections) and not smdigit_success:
        stage3_errors.append("'smdigit' failed: none of the crops had smsegs ≥ 14 and smdot ≥ 1")

    # --- tele ---
    tele_success = False
    for det in [d for d in detections if d['class'] == 'tele']:
        x1, y1, x2, y2 = det['box']
        crop = image[y1:y2+2, x1:x2+2]
        counts = run_secondary_and_count(crop)

        if counts.get('tower', 0) >= 4:
            tele_success = True
            break

    if any(d['class'] == 'tele' for d in detections) and not tele_success:
        stage3_errors.append("'tele' failed: none of the crops had towers ≥ 4")

    # --- switch ---
    switch_success = False
    for det in [d for d in detections if d['class'] == 'switch']:
        x1, y1, x2, y2 = det['box']
        crop = image[y1:y2+2, x1:x2+2]
        counts = run_secondary_and_count(crop)

        if (counts.get('swl', 0) >= 1 and
            counts.get('swr', 0) >= 1 and
            counts.get('swb', 0) >= 1):
            switch_success = True
            break

    if any(d['class'] == 'switch' for d in detections) and not switch_success:
        stage3_errors.append("'switch' failed: none of the crops had swl, swr, and swb ≥ 1")

    # --- Stage5 ---
    stage5_errors = []
    for det in detections:
        if det['class'] == 'kwarhz':
            crop = det['crop']
            h = crop.shape[0] // 2
            w = crop.shape[1] // 2
            kw = crop[:, :math.ceil(w - w*0.09)]
            a = crop[:, math.ceil(w - w*0.255):]

            kw_gray = cv2.cvtColor(kw, cv2.COLOR_BGR2GRAY)
            a_gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)

            kw_text = ''.join([res[1] for res in reader.readtext(kw_gray)]).replace(' ', '').lower()
            a_text = ''.join([res[1] for res in reader.readtext(a_gray)]).replace(' ', '').lower()

            combined_text = (kw_text + a_text).replace(' ', '')
            if 'kwarhz' not in combined_text:
                stage5_errors.append(f"OCR mismatch for 'kwarhz': got '{combined_text}'")

    # --- Stage 6: Earth underline detection using third model ---
    stage6_errors = []
    earth_model = YOLO('/content/drive/MyDrive/best_earth.pt')
    earth_detections = [d for d in detections if d['class'] == 'earth']
    earth_success = False

    for det in earth_detections:
        x1, y1, x2, y2 = det['box']
        crop = image[y1:y2+2, x1:x2+2]
        results = earth_model(crop)

        counts = Counter()
        for r in results:
            for box in r.boxes:
                class_idx = int(box.cls[0].item())
                name = earth_model.names[class_idx]
                counts[name] += 1

        if counts.get('underline', 0) >= 3:
            earth_success = True
            break

    if earth_detections and not earth_success:
        stage6_errors.append("'earth' failed: none of the crops had at least 3 underline detections")

    return {
        'stage1_errors': stage1_errors,
        'stage2_errors': stage2_errors,
        'stage3_errors': stage3_errors,
        'stage4_errors': stage4_errors,
        'stage5_errors': stage5_errors,
        'stage6_errors': stage6_errors
    }

# --- Example usage ---
if __name__ == "__main__":
    result = run_defect_detection("/content/allgood_sharpened.png")

    print("\n--- Stage 1: Missing or Count Errors ---")
    for err in result['stage1_errors']:
        print( err)

    print("\n--- Stage 2: OCR Mismatch Errors ---")
    for err in result['stage2_errors']:
        print( err)

    print("\n--- Stage 3: Model Output Errors ---")
    for err in result['stage3_errors']:
        print(err)

    print("\n--- Stage 4: 7-Segment digit Intensity Errors ---")
    for err in result['stage4_errors']:
        print( err)

    print("\n--- Stage 5: KWARHZ OCR Errors ---")
    for err in result['stage5_errors']:
        print( err)

    print("\n--- Stage 6: Earth Underline Detection Errors ---")
    for err in result['stage6_errors']:
        print(err)
